In [1]:
import os

os.environ["PYTENSOR_FLAGS"] = (
    f"base_compiledir=/scratch/{os.environ.get('USER', 'default')}/pytensor_compile"
)

# Ensure compile directory exists
compile_dir = os.environ["PYTENSOR_FLAGS"].split("base_compiledir=")[1]
os.makedirs(compile_dir, exist_ok=True)

In [2]:
import pandas as pd
import dask
import gev_bayes_utils as gevbu

from utils import city_list, gev_metric_ids
from utils import roar_data_path as project_data_path

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=3,
    processes=1,
    job_cpu=3,
    memory="10GiB",
    walltime="00:40:00",
    # Include environment variables in worker environment
    job_script_prologue=[
        f"export PYTENSOR_FLAGS=base_compiledir={compile_dir}",
        f"export PYTHONPATH=$PYTHONPATH:/storage/work/dcl5300/current_projects/conus_comparison_lafferty-etal-2024/.venv/lib/python3.12/site-packages",
    ],
)

cluster.scale(10)

from dask.distributed import Client
client = Client(cluster)
client

## Get city timeseries

In [6]:
# Run for all
for city in city_list.keys():
    for metric_id in gev_metric_ids:
        gevbu.get_city_timeseries_all(city, metric_id)

## Fit Bayesian GEV

In [5]:
# Fit info
hist_years = [1950, 2014]
future_years = [2050,2100]

stationary = True
return_periods = [100]

# Parallelize with dask delayed
delayed = []

# Loop thorugh all combos
for city in city_list.keys():
    for metric_id in ['max_tasmax', 'min_tasmin', 'max_pr']:
        for years in [hist_years, future_years]:
            tmp = dask.delayed(gevbu.fit_bayesian_gev_ensemble)(
                city=city,
                metric_id=metric_id,
                years=years,
                stationary=stationary,
                return_periods=return_periods,
                dask=False
            )
            delayed.append(tmp)

In [6]:
%%time
_ = dask.compute(*delayed)

KeyboardInterrupt: 